In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob

In [34]:
# read in all casts processed in total_df.ipynb
total_df = pd.read_csv('../data/big_df.csv')

# display the first 5 rows of the df 
total_df.head()

,Pressure (dbar),Depth (m),Temperature (°C),Conductivity (µS/cm),Specific Conductance (µS/cm),Salinity (PSS),Sound Velocity (m/s),Density (kg/m³),Latitude,Longitude,UTC Time,File Name
0,0.15,0.153151,9.153524,296.339874,433.834991,0.204679,1444.146359,999.934361,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211
1,0.45,0.458792,9.255136,295.835466,431.811837,0.203774,1444.561198,999.927172,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211
2,0.75,0.764653,9.132318,295.647002,433.089545,0.204300,1444.069489,999.938574,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211
3,1.05,1.070511,9.033097,296.121842,435.049797,0.205173,1443.672646,999.948268,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211
4,1.35,1.376366,8.925369,296.570547,437.092585,0.206079,1443.240202,999.958485,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211


In [85]:
total_df.to_csv('../data/total_df.csv')

In [35]:
# Extract the first row for each cast (based on File Name)
cast_locations = total_df.groupby("File Name").first().reset_index()

# Optional: if you want only lat/lon and filename
cast_locations = cast_locations[["Latitude", "Longitude", "File Name"]]

In [36]:
cast_locations

,Latitude,Longitude,File Name
0,40.274148,-106.839900,CC2435009_20250513_181211
1,40.276507,-106.841752,CC2435009_20250513_181617
2,40.279905,-106.851733,CC2435009_20250513_182530
3,40.279532,-106.852019,CC2435009_20250513_182851
4,40.283628,-106.853450,CC2435009_20250513_194404
...,...,...,...
161,40.274410,-106.850507,CC2435009_20250519_182038
162,40.274005,-106.850038,CC2435009_20250519_182316
163,40.283275,-106.854001,CC2435009_20250519_185748
164,40.283448,-106.853752,CC2435009_20250519_190629


In [5]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import folium


# Create map centered at mean lat/lon
m = folium.Map(location=[
    cast_locations["Latitude"].mean(),
    cast_locations["Longitude"].mean()
], zoom_start=13)

# Add cast locations as circle markers
for _, row in cast_locations.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        popup=row["File Name"],
        color="blue",
        fill=True,
        fill_color="blue"
    ).add_to(m)

# Save and/or display
m.save("stagecoach_casts_map.html")
m  # Will display inline in Jupyter


In [84]:
import pandas as pd
from shapely.geometry import Point, Polygon
import folium
from folium import Element
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from base64 import b64encode

# ---- Define Polygons ----
polygon_coords = {
    'DOC': [(40.28429, -106.85365), (40.28360, -106.85341), (40.28240, -106.85488), (40.28356, -106.85524)],
    'HAR': [(40.28424, -106.85348), (40.28565, -106.85185), (40.28797, -106.85680), (40.28793, -106.85836)],
    'MOR': [(40.28003, -106.84560), (40.28021, -106.84274), (40.27252, -106.83760), (40.27183, -106.84056)],
    'GLZ': [(40.27291, -106.84515), (40.27610, -106.85043), (40.27509, -106.85397), (40.27129, -106.84970)],
    'KEY': [(40.27705, -106.86361), (40.27700, -106.86697), (40.28124, -106.86839), (40.28180, -106.86597)],
    'LIL': [(40.27265, -106.86163), (40.27315, -106.85961), (40.27114, -106.85906), (40.27115, -106.86068)],
    'ASP': [(40.272286, -106.876870),(40.272286, -106.876599),(40.272016, -106.876599),(40.272016, -106.876870)]
}
polygons = {name: Polygon(coords) for name, coords in polygon_coords.items()}

# ---- Classify Points by Area ----
def get_area(lat, lon):
    point = Point(lat, lon)
    for name, poly in polygons.items():
        if poly.contains(point):
            return name
    return 'MID'

# ---- Process Data ----
cast_locations['Area'] = cast_locations.apply(lambda row: get_area(row['Latitude'], row['Longitude']), axis=1)
total_df['Area'] = total_df.apply(lambda row: get_area(row['Latitude'], row['Longitude']), axis=1)
cast_locations['Date'] = pd.to_datetime(cast_locations['File Name'].str.extract(r'_(\d{8})_')[0], format='%Y%m%d')

# ---- Color Map for Areas ----
area_colors = {
    'DOC': 'red',
    'HAR': 'cyan',
    'MOR': 'purple',
    'GLZ': 'orange',
    'KEY': 'darkblue',
    'LIL': 'darkred',
    'ASP': 'pink',
    'MID': 'gray'
}

# ---- Legend HTML ----
legend_html = """
<div style="
    position: fixed; 
    bottom: 30px; left: 30px; width: 150px; z-index:500;
    background-color: white; padding: 10px; border: 2px solid grey;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3); font-size:14px;">
<b>Areas</b><br>
<i style='background:red; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Dock<br>
<i style='background:cyan; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Harding Cove<br>
<i style='background:purple; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Morrison Cove<br>
<i style='background:orange; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Glizzy Cove<br>
<i style='background:darkblue; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Keystone Cove<br>
<i style='background:darkred; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Small Inlet<br>
<i style='background:pink; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Aspen View<br>
<i style='background:gray; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Middle of Reservoir
</div>
"""

# ---- Widget Setup ----
date_picker = widgets.DatePicker(description='Pick a date', value=cast_locations['Date'].min())
toggle_button = widgets.ToggleButton(value=False, description="Show All Data", button_style='info')
map_output = widgets.Output()

def update_map(change=None):
    with map_output:
        clear_output(wait=True)
        if toggle_button.value:
            filtered = cast_locations
        else:
            selected_date = date_picker.value
            filtered = cast_locations[cast_locations['Date'] == pd.to_datetime(selected_date)]
        
        # Create map
        m = folium.Map(
            location=[filtered["Latitude"].mean(), filtered["Longitude"].mean()],
            zoom_start=13
        )

        
        # Plot points
        for _, row in filtered.iterrows():
            with open(row['ImagePath'], 'rb') as f:
                encoded = b64encode(f.read()).decode()

            popup_html = f"""
                <div>
                    <a href="data:image/png;base64,{encoded}" target="_blank">
                        <img src="data:image/png;base64,{encoded}" width="700">
                    </a><br>
                    <b>Date:</b> {row['Date'].date()}<br>
                    <b>File:</b> {row['File Name']}<br>
                    <b>Lat:</b> {row['Latitude']}<br>
                    <b>Lon:</b> {row['Longitude']}<br>
                    <b>Area:</b> {row['Area']}
                </div>
            """

    
            iframe = folium.IFrame(html=popup_html, width=450, height=300)
            popup = folium.Popup(iframe, max_width=450)    
            
            folium.CircleMarker(
                location=[row["Latitude"], row["Longitude"]],
                radius=5,
                popup = popup,
                color=area_colors.get(row["Area"], "gray"),
                fill=True,
                fill_color=area_colors.get(row["Area"], "gray"),
                fill_opacity=0.9
            ).add_to(m)

        m.get_root().html.add_child(Element(legend_html))
        display(m)

# ---- Connect Handlers ----
date_picker.observe(update_map, names='value')
toggle_button.observe(update_map, names='value')

# ---- Show UI ----
display(widgets.HBox([date_picker, toggle_button]))
display(map_output)
update_map()  # Initial call


Output()

In [26]:
cast_locations[cast_locations['Area'] == 'DOC']

,Latitude,Longitude,File Name,Area,Date
4,40.283628,-106.853450,CC2435009_20250513_194404,DOC,2025-05-13
5,40.283628,-106.853445,CC2435009_20250513_194520,DOC,2025-05-13
6,40.283271,-106.853989,CC2435009_20250513_194839,DOC,2025-05-13
7,40.283283,-106.854040,CC2435009_20250513_195019,DOC,2025-05-13
8,40.283442,-106.853767,CC2435009_20250513_195323,DOC,2025-05-13
...,...,...,...,...,...
144,40.284132,-106.853688,CC2435009_20250519_165212,DOC,2025-05-19
145,40.284114,-106.853663,CC2435009_20250519_165532,DOC,2025-05-19
163,40.283275,-106.854001,CC2435009_20250519_185748,DOC,2025-05-19
164,40.283448,-106.853752,CC2435009_20250519_190629,DOC,2025-05-19


In [69]:
import os

output_dir = "profiles"
os.makedirs(output_dir, exist_ok=True)

grouped_meas = total_df.groupby('File Name')

image_paths = {}

for date, group in grouped_meas:
    group = group.sort_values(by='Depth (m)') 

    fig, axs = plt.subplots(1, 4, figsize=(20, 6), sharey=True)
    fig.suptitle(f'Profiles on {date}', fontsize=36)

    axs[0].plot(group['Temperature (°C)'], group['Depth (m)'])
    axs[0].set_xlabel('Temperature (°C)')
    axs[0].set_ylabel('Depth (m)')
    axs[0].invert_yaxis()

    axs[1].plot(group['Conductivity (µS/cm)'], group['Depth (m)'])
    axs[1].set_xlabel('Conductivity (µS/cm)')

    axs[2].plot(group['Salinity (PSS)'], group['Depth (m)'])
    axs[2].set_xlabel('Salinity (PSS)')

    axs[3].plot(group['Density (kg/m³)'], group['Depth (m)'])
    axs[3].set_xlabel('Density (kg/m³)')

    plt.tight_layout()

    img_path = os.path.join(output_dir, f"profile_{date}.png")
    fig.savefig(img_path)
    plt.close(fig)

    # Store path and representative coordinates (first row)
    rep_lat = group.iloc[0]['Latitude']
    rep_lon = group.iloc[0]['Longitude']
    image_paths[date] = (img_path, rep_lat, rep_lon)


# Step 1: Create a DataFrame from the dictionary
image_df = pd.DataFrame.from_dict(image_paths, orient='index', columns=['ImagePath', 'Latitude_dummy', 'Longitude_dummy'])

# Step 2: Reset the index to get the keys (file names) as a column
image_df.reset_index(inplace=True)
image_df.rename(columns={'index': 'File Name'}, inplace=True)

# Step 3: Merge with your existing cast_locations DataFrame on 'File Name'
cast_locations = cast_locations.merge(image_df[['File Name', 'ImagePath']], on='File Name', how='left')     

In [46]:
# Step 1: Create a DataFrame from the dictionary
image_df = pd.DataFrame.from_dict(image_paths, orient='index', columns=['ImagePath', 'Latitude_dummy', 'Longitude_dummy'])

# Step 2: Reset the index to get the keys (file names) as a column
image_df.reset_index(inplace=True)
image_df.rename(columns={'index': 'File Name'}, inplace=True)

# Step 3: Merge with your existing cast_locations DataFrame on 'File Name'
cast_locations = cast_locations.merge(image_df[['File Name', 'ImagePath']], on='File Name', how='left')